**PROJECT: Online Retail Sales EDA**

**PROBLEM STATEMENT**

In this project, I am performing Exploratory Data Analysis (EDA) on a real-world E-commerce retail dataset. My goal is to analyze customer purchasing behavior, identify top-selling products, uncover seasonal trends, and segment customers based on their buying patterns. As part of the process, I am also handling common data preprocessing tasks such as dealing with missing values, removing duplicate entries, and managing outliers.

**IMPORT LIBRARIES AND LOAD DATA**

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

**DATA EXPLORATION**

In [101]:
# LOAD THE DATASET

sales_df = pd.read_excel('src/online_retail_II.xlsx')
sales_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [102]:
# Data Overview
print("Data Overview:")
print(sales_df.info())

print("\nData Statistics:")
print(sales_df.describe())

# check missing values
print("\nMissing values in each column:")
print(sales_df.isnull().sum())

Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB
None

Data Statistics:
            Quantity                    InvoiceDate          Price  \
count  525461.000000                         525461  525461.000000   
mean       10.337667  2010-06-28 11:37:36.845017856       4.688834   
min     -9600.000000            2009-12-01 07:45:00  -53594.

**Initial Observation**

- There are **525461** entries(rows/products) and **8** columns(attributes), with missing values in **Description(2928)** and **Customer ID(107927)**.
- There are some negative and extreme values in **Quantity** and **Price** indicating outliers.
- 3 datatypes **object**,**integer**, and **dateTime**.

**DATA CLEANING**

In [ ]:
# Handle Duplicates 

# check for duplicates 
print(f"\nNumber of duplicate rows: {sales_df.duplicated().sum()}")

# drop duplicates
sales_df = sales_df.drop_duplicates()

# check for duplicates again
print(f"\nNumber of duplicate rows after dropping: {sales_df.duplicated().sum()}")\


Number of duplicate rows: 6865

Number of duplicate rows after dropping: 0
<class 'pandas.core.frame.DataFrame'>
Index: 518596 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      518596 non-null  object        
 1   StockCode    518596 non-null  object        
 2   Description  515668 non-null  object        
 3   Quantity     518596 non-null  int64         
 4   InvoiceDate  518596 non-null  datetime64[ns]
 5   Price        518596 non-null  float64       
 6   Customer ID  410763 non-null  float64       
 7   Country      518596 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 35.6+ MB


In [ ]:
# check for cancelled orders
print(sales_df['Invoice'].astype(str).str.startswith('C').value_counts())
print("\nOriginal rows:", sales_df.shape[0])

# Remove cancelled orders (where Invoice starts with 'C')
sales_df = sales_df[~sales_df['Invoice'].astype(str).str.startswith('C')]
print("After removing cancelled orders:", sales_df.shape[0])

# Remove negative/zero quantity and price
sales_df = sales_df[(sales_df['Quantity'] > 0) & (sales_df['Price'] > 0)]
print("After removing outliers:", sales_df.shape[0])

Invoice
False    508414
True      10182
Name: count, dtype: int64
Original rows: 518596
After removing cancelled orders: 508414
After removing outliers: 504730


In [107]:
# Handle missing values

# Drop rows with missing descriptions
sales_df = sales_df.dropna(subset=['Description'])

# For general sales/product analysis, keeping all data 
sales_df_all = sales_df.copy()

# For customer-based analysis , we drop rows where 'Customer ID' is missing
sales_df_customers = sales_df.dropna(subset=['Customer ID'])

# check again for missing values
print("\nMissing values after handling:")
print(f"All\n{sales_df_all.isnull().sum()}")
print(f"\nCustomer-based Analysis Only - \n{sales_df_customers.isnull().sum()}")

# Check total values after cleaning
print(f"\nTotal rows after cleaning(All):{sales_df_all.shape[0]}")
print(f"Total rows after cleaning(Customer-based):{sales_df_customers.shape[0]}")


Missing values after handling:
All
Invoice             0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
Price               0
Customer ID    103814
Country             0
dtype: int64

Customer-based Analysis Only - 
Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

Total rows after cleaning(All):504730
Total rows after cleaning(Customer-based):400916


**INSIGHTS (AFTER CLEANING)**

- **6865 duplicate rows** were identified and removed to ensure data accuracy.
- **Cancelled orders** (Invoice starting with "C") totaling **10,182** rows were excluded.
- All rows with **negative or zero Quantity and Price** were removed to eliminate invalid and outlier transactions.
- A total of **13,866** rows were dropped during outlier and cancelled order removal, reducing the dataset to **504,730** rows.
- Rows with missing **Description (product name)** were dropped, as product-level insights require product identification.
- For general analysis, rows with missing **Customer ID (103,814 rows)** were retained.
- For customer-based segmentation (like **RFM analysis**), only rows with valid **Customer IDs** were used, leaving **400,916** rows in **sales_df_customers**.
- Final cleaned dataset:
    - **sales_df_all** → **504,730** rows for overall product and sales trends.
    - **sales_df_customers** → **400,916 rows** for accurate customer-level insights.